# 🌦️ Analyse Météo – Pipeline API → S3 → DuckDB

Ce notebook analyse les données météo issues de la mini-pipeline :

- Ingestion via **API Open-Meteo**
- Stockage dans **S3 (Landing → Staging → Refined / Parquet)**
- Chargement dans **DuckDB (warehouse/weather.duckdb)**
- Analyse et visualisation avec **Pandas / Matplotlib / Seaborn**

> Ce notebook se place **en bout de chaîne**, en lisant directement la table `weather` dans DuckDB.


In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Affichage inline pour Jupyter
%matplotlib inline

plt.style.use("default")

In [ ]:
# 📁 Connexion au warehouse DuckDB
DB_PATH = Path("warehouse/weather.duckdb")

if not DB_PATH.exists():
    raise FileNotFoundError(f"Base DuckDB introuvable : {DB_PATH}.\n"
                            "Assure-toi d'avoir lancé la pipeline complète avant (run_pipeline.py).")

con = duckdb.connect(str(DB_PATH))
con

In [ ]:
# 🔍 Aperçu de la table weather
df = con.execute("SELECT * FROM weather LIMIT 10").df()
df

In [ ]:
# 📊 Dimensions & types
print("Shape :", df.shape)

schema = con.execute("DESCRIBE weather").df()
schema

In [ ]:
# ⚠️ Ici, on charge toutes les données en mémoire.
# OK pour un petit dataset météo comme le nôtre.
weather = con.execute("SELECT * FROM weather").df()

weather['date'] = pd.to_datetime(weather['date'])
weather.sort_values('date', inplace=True)

weather.head()

In [ ]:
# 📈 Statistiques descriptives globales
weather.describe(include='all')

In [ ]:
# 🌍 Nombre de points par ville
city_counts = weather['city'].value_counts()
city_counts

In [ ]:
plt.figure(figsize=(6,4))
city_counts.plot(kind='bar')
plt.title("Nombre de mesures par ville")
plt.xlabel("Ville")
plt.ylabel("Nombre de points")
plt.tight_layout()
plt.show()

In [ ]:
# 📉 Température au fil du temps par ville
plt.figure(figsize=(10,6))
for city, sub in weather.groupby('city'):
    plt.plot(sub['date'], sub['temperature_c'], label=city, alpha=0.7)

plt.title("Évolution horaire de la température par ville")
plt.xlabel("Date")
plt.ylabel("Température (°C)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=weather, x='city', y='temperature_c')
plt.title("Distribution des températures par ville")
plt.xlabel("Ville")
plt.ylabel("Température (°C)")
plt.tight_layout()
plt.show()

In [ ]:
# 🌩️ Analyse des épisodes de 'storm'
storm_rate = weather.groupby('city')['is_storm'].mean().sort_values(ascending=False)

print("Taux d'épisodes storm par ville :")
display(storm_rate)

plt.figure(figsize=(6,4))
storm_rate.plot(kind='bar')
plt.title("Taux d'épisodes venteux/pluvieux par ville")
plt.xlabel("Ville")
plt.ylabel("Proportion de lignes avec is_storm=1")
plt.tight_layout()
plt.show()

In [ ]:
# 🔗 Corrélations entre variables numériques
numeric_cols = ['temperature_c', 'humidity', 'wind_kmh', 'precip_mm', 'is_storm']
corr = weather[numeric_cols].corr()

plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation (variables météo)")
plt.tight_layout()
plt.show()

## 🧩 Synthèse & Pistes d'Insights

Quelques exemples de lecture métier possibles :

- **Température moyenne par ville** : permet de comparer les profils climatiques des différentes zones.
- **Taux d'épisodes `is_storm = 1`** par ville : intéressant pour repérer les zones plus exposées à des conditions météo difficiles.
- **Corrélation humidité / précipitations / vents** : utile pour comprendre les patterns météo qui peuvent impacter un service (pannes, incidents réseau, consommation).

Dans un contexte Orange, on pourrait par exemple :

- Croiser ces données météo avec des indicateurs réseau (tickets, incidents, churn localisé, etc.).
- Construire des features météo pour enrichir un modèle de prédiction (pannes, trafic, consommation, churn).

Ce notebook reste volontairement simple, mais la structure (DuckDB + Pandas + Viz) est prête pour être étendue.
